In [ ]:
import tensorflow as tf
from skimage import io
import matplotlib.pyplot as plt
import random
import os
import cv2
import numpy as np

def image_to_patches(image, patch_height, patch_width):
    # resize image so that it's dimensions are dividable by patch_height and patch_width
    image_height = tf.cast(tf.shape(image)[0], dtype=tf.float32)
    image_width = tf.cast(tf.shape(image)[1], dtype=tf.float32)
    height = tf.cast((tf.ceil(image_height / patch_height)-1) * patch_height, dtype=tf.int32)
    width = tf.cast((tf.ceil(image_width / patch_width) - 1) * patch_width, dtype=tf.int32)

    num_rows = height // patch_height
    num_cols = width // patch_width
    # make zero-padding
    image = tf.squeeze(tf.image.resize_image_with_crop_or_pad(image, height, width))

    # get slices along the 0-th axis
    image = tf.reshape(image, [num_rows, patch_height, width, -1])
    # h/patch_h, w, patch_h, c
    image = tf.transpose(image, [0, 2, 1, 3])
    # get slices along the 1-st axis
    # h/patch_h, w/patch_w, patch_w,patch_h, c
    image = tf.reshape(image, [num_rows, num_cols, patch_width, patch_height, -1])
    # num_patches, patch_w, patch_h, c
    image = tf.reshape(image, [num_rows * num_cols, patch_width, patch_height, -1])
    # num_patches, patch_h, patch_w, c
    return tf.transpose(image, [0, 2, 1, 3])

def get_random_n_patches(patches, k):
    n = patches.shape[0]
    if n < 2:
        return patches
    while(True):
        if(k <= n):
            rand = random.sample(range(n), k)
            break
        else:
            k -= 3
    return patches[rand]

def patch_img_to_file(base_path, file_name, dest, label_anno):
    global data
    global label
    image = io.imread(os.path.join(base_path, file_name))
    #print(file_name)
    tile_size = 512
    image = tf.constant(image)
    tiles = image_to_patches(image, tile_size, tile_size)
    sess = tf.Session()
    I, tiles = sess.run([image, tiles])
#     print(I.shape)  #anh goc
#     print(tiles.shape)
    tiles = get_random_n_patches(tiles, 5)
    j = 1
    for i, tile in enumerate(tiles):
#         tmp = os.path.splitext(file_name)
#         new_name = tmp[0] + "_patch" + str(j) + tmp[1]
#         cv2.imwrite(os.path.join(dest, new_name), tile)
#         j+=1
        data.append(tile)
        label.append(label_anno)
BASE1 = "/home/liem/hai/recapture_classification/training_data/natural"
DES1 = "/home/liem/hai/recapture_classification/training_data/natural_patch"
BASE2 = "/home/liem/hai/recapture_classification/training_data/recapture"
DES2 = "/home/liem/hai/recapture_classification/training_data/recapture_patch"
LOG0 = "/home/liem/hai/recapture_classification/np0"
LOG1 = "/home/liem/hai/recapture_classification/np1"
data = []
label = []
cnt1 = 1
cnt2 = 1
total = 1
for item in os.listdir(BASE1):
    patch_img_to_file(BASE1, item, DES1, 0)
    if(cnt1 % 100 == 0):
        np_data = np.asarray(data)
        np_label = np.asarray(label)
        name_data = "data_natural_" + str(cnt1)
        name_label = "label_natural_" + str(cnt1)
        np.save(os.path.join(LOG0, name_data), np_data)
        np.save(os.path.join(LOG0, name_label), np_label)
        data = []
        label = []
        print("saved at number " + str(cnt1) + "--natural images--")
    cnt1+=1
    total +=1
print("******************DONE-phase 1******************************")
data = []
label = []
for item in os.listdir(BASE2):
    patch_img_to_file(BASE2, item, DES2, 1)
    if(cnt2 % 75 == 0):
        np_data = np.asarray(data)
        np_label = np.asarray(label)
        name_data = "data_recapture_" + str(cnt2)
        name_label = "label_recapture_" + str(cnt2)
        np.save(os.path.join(LOG1, name_data), np_data)
        np.save(os.path.join(LOG1, name_label), np_label)
        data = []
        label = []
        print("saved at number " + str(cnt2) + "--recapture images--")
    cnt2+=1
    total +=1
print("total_image: " + str(total))
# data = np.asarray(data)
# label = np.asarray(label)
# print(data.shape)
# print(label.shape)
# np.save("data", data)
# np.save("label", label)

saved at number 100--natural images--
